# Deploy APIM for Service Fabric Managed Cluster
This notebook walks through the PowerShell commands to deploy and configure APIM to work with a Service Fabric Managed Cluster.

In [ ]:
# Step 1: Create Resource Group
$resourceGroupName = 'TestRG'
$location = 'EastUS'
New-AzResourceGroup -Name $resourceGroupName -Location $location

In [ ]:
# Step 2: Create Network Security Group (NSG)
$networkSecurityGroupName = 'vnet-apim-nsg'
$networkSecurityGroup = New-AzNetworkSecurityGroup -Name $networkSecurityGroupName -ResourceGroupName $resourceGroupName -Location $location

In [ ]:
# Step 3: Configure NSG rules for APIM
Add-AzNetworkSecurityRuleConfig -Name 'AllowManagementEndpoint' -NetworkSecurityGroup $networkSecurityGroup -Description 'Management endpoint for Azure portal and PowerShell' -Access Allow -Protocol Tcp -Direction Inbound -Priority 300 -SourceAddressPrefix ApiManagement -SourcePortRange * -DestinationAddressPrefix VirtualNetwork -DestinationPortRange 3443
# Update the NSG
Set-AzNetworkSecurityGroup -NetworkSecurityGroup $networkSecurityGroup

In [ ]:
# Step 4: Create Virtual Network (VNET)
$vnet = @{ Name = 'VNet'; ResourceGroupName = $resourceGroupName; Location = $location; AddressPrefix = '10.0.0.0/16' }
$virtualNetwork = New-AzVirtualNetwork @vnet

In [ ]:
# Step 5: Create subnet configurations
$sfmcSubnet = @{ Name = 'sfmc'; VirtualNetwork = $virtualNetwork; AddressPrefix = '10.0.0.0/24' }
$apimSubnet = @{ Name = 'apim'; VirtualNetwork = $virtualNetwork; AddressPrefix = '10.0.1.0/24'; NetworkSecurityGroup = $networkSecurityGroup }
Add-AzVirtualNetworkSubnetConfig @sfmcSubnet
Add-AzVirtualNetworkSubnetConfig @apimSubnet

In [ ]:
# Step 6: Associate the subnet configuration to the virtual network
$virtualNetwork | Set-AzVirtualNetwork

In [ ]:
# Step 7: Gather Service Fabric Managed Cluster requirements
# Enumerate SFRP principals
$sfrpPrincipals = @(Get-AzADServicePrincipal -DisplayName 'Azure Service Fabric Resource Provider')

# Obtain the Subnet Resource Id for SFMC
$virtualNetwork = Get-AzVirtualNetwork -Name $vnet.Name -ResourceGroupName $resourceGroupName
$sfmcSubnetID = $virtualNetwork.Subnets | Where-Object Name -eq $sfmcSubnet.Name | Select-Object -ExpandProperty Id

# Assign Network Contributor role to each SFRP principal
foreach($sfrpPrincipal in $sfrpPrincipals) {
    New-AzRoleAssignment -PrincipalId $sfrpPrincipal.Id -RoleDefinitionName 'Network Contributor' -Scope $sfmcSubnetID
}

In [ ]:
# Step 8: Create Public IP Address (for APIM)
$domainNameLabel = 'apimip'
$ip = @{ Name = 'apimip'; ResourceGroupName = $resourceGroupName; Location = $location; Sku = 'Standard'; AllocationMethod = 'Static'; IpAddressVersion = 'IPv4'; DomainNameLabel = $domainNameLabel }
New-AzPublicIpAddress @ip

In [ ]:
# Step 9: Create API Management Service with external VNET integration
$virtualNetwork = Get-AzVirtualNetwork -Name $vnet.Name -ResourceGroupName $resourceGroupName
$apimSubnetId = $virtualNetwork.Subnets | Where-Object Name -eq $apimSubnet.Name | Select-Object -ExpandProperty Id
$apimNetwork = New-AzApiManagementVirtualNetwork -SubnetResourceId $apimSubnetId
$publicIpAddressId = (Get-AzPublicIpAddress -Name $ip.Name -ResourceGroupName $resourceGroupName).Id
$apimName = 'myApimCloud'
$adminEmail = 'admin@contoso.com'
$organization = 'contoso'

New-AzApiManagement -ResourceGroupName $resourceGroupName -Location $location -Name $apimName -Organization $organization -AdminEmail $adminEmail -VirtualNetwork $apimNetwork -VpnType 'External' -Sku 'Developer' -PublicIpAddressId $publicIpAddressId

In [ ]:
# Step 10: Deploy the Service Fabric Managed Cluster
$templateFile = "$pwd\sfmc-template.json"
$adminPassword = '<enter a password>'
$clientCertificateThumbprint = '<enter a thumbprint>'

$sfmc = @{ 
    clusterName = 'sfmcapim';
    clusterSku = 'Standard';
    adminUserName = 'cloudadmin';
    adminPassword = $adminPassword;
    clientCertificateThumbprint = $clientCertificateThumbprint;
    nodeType1name = 'nodetype1';
    nodeType1vmSize = 'Standard_D2s_v3';
    nodeType1vmInstanceCount = 5;
    nodeType1dataDiskSizeGB = 256;
    nodeType1vmImagePublisher = 'MicrosoftWindowsServer';
    nodeType1vmImageOffer = 'WindowsServer';
    nodeType1vmImageSku = '2022-Datacenter';
    nodeType1vmImageVersion = 'latest';
    subnetId = $sfmcSubnetID
}

New-AzResourceGroupDeployment -Name 'sfmcDeployment' -ResourceGroupName $resourceGroupName -TemplateFile $templateFile -TemplateParameterObject $sfmc

In [ ]:
# Step 12: Create a system-assigned managed identity for APIM
$apimService = Get-AzApiManagement -ResourceGroupName $resourceGroupName -Name $apimName
Set-AzApiManagement -InputObject $apimService -SystemAssignedIdentity

In [ ]:
# Step 13: Configure new Key vault Access policy using managed identity
$keyVaultName = 'apimKV'
$managedIdentityId = (Get-AzADServicePrincipal -SearchString $apimName).Id
Set-AzKeyVaultAccessPolicy -VaultName $keyVaultName -ObjectId $managedIdentityId  -PermissionsToSecrets get,list

In [ ]:
# Step 14: Create an Azure Key Vault certificate in APIM
$kvcertId = 'apimcloud-com'
$secretIdentifier = 'https://apimKV.vault.azure.net/secrets/apimcloud-com/xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
$apiMgmtContext = New-AzApiManagementContext -ResourceGroupName $resourceGroupName -ServiceName $apimName

$keyvault = New-AzApiManagementKeyVaultObject -SecretIdentifier $secretIdentifier
$keyVaultCertificate = New-AzApiManagementCertificate -Context $apiMgmtContext -CertificateId $kvcertId -KeyVault $keyvault

In [ ]:
# Step 15: Create a Service Fabric backend in APIM using the certificate
$serviceFabricAppUrl = 'fabric:/sfWeatherApiCore/WeatherApi'
$clusterName = 'sfmcapim'
$clusterResource = Get-AzResource -Name $clusterName -ResourceType 'Microsoft.ServiceFabric/managedclusters'
$cluster = Get-AzServiceFabricManagedCluster -Name $clusterName -ResourceGroupName $clusterResource.ResourceGroupName
$serverCertThumbprint = $clusterResource.Properties.clusterCertificateThumbprints
$x509CertName = $cluster.ClusterId.Replace('-', '')

$backend = @{ 
    apimName = $apimName;
    backendName = 'ServiceFabricBackend';
    description = 'Service Fabric backend';
    clientCertificateThumbprint = $keyVaultCertificate.Thumbprint;
    managementEndpoints = @("https://$($cluster.Fqdn):$($cluster.HttpGatewayConnectionPort)");
    maxPartitionResolutionRetries = 5;
    serviceFabricManagedClusterFqdn = $cluster.Fqdn;
    protocol = 'http';
    url = $serviceFabricAppUrl;
    validateCertificateChain = $false;
    validateCertificateName = $false
}

$backend | ConvertTo-Json | Write-Output

New-AzResourceGroupDeployment -Name 'apimBackendDeployment' -ResourceGroupName $resourceGroupName -TemplateFile "$pwd\apim-backend.json" -TemplateParameterObject $backend

In [ ]:
# Step 16: Create an API in APIM
$apiId = 'service-fabric-weatherforecast-app'
$apiName = 'Service Fabric WeatherForecast App'
$serviceUrl = 'http://servicefabric'
New-AzApiManagementApi -Context $apiMgmtContext -ApiId $apiId -Name $apiName -ServiceUrl $serviceUrl -Protocols @('http', 'https') -Path 'api'

In [ ]:
# Step 17: Create an Operation for the API
$operationId = 'service-fabric-weatherforecast-app-operation'
$operationName = 'Service Fabric WeatherForecast App Operation'
New-AzApiManagementOperation -Context $apiMgmtContext -ApiId $apiId -OperationId $operationId -Name $operationName -Method 'GET' -UrlTemplate '' -Description ''

In [ ]:
# Step 18: Create a Policy for the API
$sfResolveCondition = '@((int)context.Response.StatusCode != 200)'
$policyString = "
<policies>
    <inbound>
        <base />
        <set-backend-service backend-id=`"ServiceFabricBackend`" sf-resolve-condition=`"$sfResolveCondition`" sf-service-instance-name=`"$serviceFabricAppUrl`" />
    </inbound>
    <backend>
        <base />
    </backend>
    <outbound>
        <base />
    </outbound>
    <on-error>
        <base />
    </on-error>
</policies>"

Set-AzApiManagementPolicy -Context $apiMgmtContext -ApiId $apiId -Policy $policyString -Format 'application/vnd.ms-azure-apim.policy.raw+xml'